# AKS Cookbook

## 🧪 Azure Service Operator on AKS

![visual](visual.png)

Azure Service Operator (ASO) allows you to deploy and maintain a wide variety of Azure Resources using the Kubernetes tooling you already know and use.

Instead of deploying and managing your Azure resources separately from your Kubernetes application, ASO allows you to manage them together, automatically configuring your application as needed.
For example, ASO can set up your Redis Cache or PostgreSQL database server and then configure your Kubernetes application to use them.

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Set your Azure Subscription](#1)
- [2️⃣ Get AKS credentials and verify the connection](#2)
- [3️⃣ Install and verify cert-manager on the cluster](#3)
- [4️⃣ Create a Service Principal](#4)
- [5️⃣ Create the Azure Service Operator namespaced secret](#5)
- [6️⃣ Install latest v2+ Helm ASO chart and include the CRDs](#6)
- [7️⃣ Create a new Azure Resource with ASO](#7)
- [🗑️ Clean up resources](#clean)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Provide either the tenant and the subscription IDs, before running the lab.
- Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management).
- Check your aks cluster name. If you want to create a new AKS cluster, please refer to this [Jupyter notebook](new-aks-cluster.ipynb)

In [ ]:
import os

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"
tenant_id = "<tenant_id>"
subscription_id = "<subscription_id>"
aks_resource_name = "aks-service-operator"
cert_manager_yaml = "https://github.com/jetstack/cert-manager/releases/download/v1.14.1/cert-manager.yaml"
crd_pattern = "resources.azure.com/ResourceGroup"

<a id='1'></a>
### 1️⃣ Set your Azure Subscription
Use this step to specify which Azure tenant and subscription should be used for subsequent Azure CLI commands.

In [ ]:
! az login --tenant {tenant_id}
! az account set --subscription {subscription_id}

<a id='2'></a>
### 2️⃣ Get AKS credentials and verify the connection.
You need kubelogin credential plugin installed - see https://github.com/Azure/kubelogin.

In [ ]:
! az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name} --overwrite-existing

<a id='3'></a>
### 3️⃣ Install and verify cert-manager on the cluster.
cert-manager is a powerful and extensible X.509 certificate controller for Kubernetes and OpenShift workloads. It will obtain certificates from a variety of Issuers, both popular public Issuers as well as private Issuers, and ensure the certificates are valid and up-to-date, and will attempt to renew certificates at a configured time before expiry.

In [ ]:
! kubectl apply -f {cert_manager_yaml}

# Check the cert-manager pods have started successfully before proceeding.
! kubectl get pods -n cert-manager

<a id='4'></a>
### 4️⃣ Create a Service Principal
This identity or service principal will be used by ASO to authenticate with Azure. You'll need this to grant the identity or Service Principal permissions to create resources in your subscription.

In [ ]:
import re  
import json

# Function to run shell commands  
def run_command(command):
    print(f"Running command: {command}")
    try:
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        return True, output.decode("utf-8")  
    except subprocess.CalledProcessError as e:  
        return False, e.output.decode("utf-8")

def extract_json_from_string(output_str):  
    # Define the regular expression pattern to match JSON objects  
    json_pattern = r'\{[\s\S]*\}'
    
    # Search for the JSON object in the string  
    match = re.search(json_pattern, output_str)
      
    if match:
        json_str = match.group(0)
        try:
            json_data = json.loads(json_str)
            return json_data  
        except json.JSONDecodeError as e:  
            print(f"Failed to parse JSON. Error: {e}")  
            return None  
    else:  
        print("No JSON object found in the input string.")  
        return None  

# Create a service principal  
success, sp_output = run_command(f"az ad sp create-for-rbac -n azure-service-operator --role contributor --scopes /subscriptions/{subscription_id}")

if success:
    sp_data = extract_json_from_string(sp_output)  
    
    print("Service principal created successfully.", sp_data)
    
    # Extract the appId and password
    client_id = sp_data["appId"]
    client_secret = sp_data["password"]

    # Export the environment variables
    os.environ["AZURE_CLIENT_ID"] = client_id
    os.environ["AZURE_CLIENT_SECRET"] = client_secret
    os.environ["AZURE_SUBSCRIPTION_ID"] = subscription_id
    os.environ["AZURE_TENANT_ID"] = tenant_id
else:  
    print("Failed to create service principal. Error:", sp_output)

<a id='5'></a>
### 5️⃣ Create the Azure Service Operator namespaced secret
The secret must be named **aso-credential** and be created in the namespace you’d like to create ASO resources in.

In [ ]:
import os
import subprocess

# Ensure the environment variables are set  
required_vars = ["AZURE_SUBSCRIPTION_ID", "AZURE_TENANT_ID", "AZURE_CLIENT_ID", "AZURE_CLIENT_SECRET"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"One or more required environment variables are not set: {', '.join(missing_vars)}")
else:
    # Read the secret template
    with open("secret.yaml", "r") as file:
        secret_template = file.read()

    # Replace placeholders with actual values
    secret_content = secret_template.replace("$AZURE_SUBSCRIPTION_ID", os.getenv("AZURE_SUBSCRIPTION_ID")).replace("$AZURE_TENANT_ID", os.getenv("AZURE_TENANT_ID")).replace("$AZURE_CLIENT_ID", os.getenv("AZURE_CLIENT_ID")).replace("$AZURE_CLIENT_SECRET", os.getenv("AZURE_CLIENT_SECRET"))

    # Write the updated content to a new file
    with open("secret_applied.yaml", "w") as file:
        file.write(secret_content)

    # Apply the secret using kubectl
    apply_command = "kubectl apply -f secret_applied.yaml"
    result = subprocess.run(apply_command, shell=True, capture_output=True, text=True)

    # Print the result
    if result.returncode == 0:
        print("Secret applied successfully.")
        print(result.stdout)
    else:
        print("Failed to apply the secret.")
        print(result.stderr)
        
! kubectl get pods -n azureserviceoperator-system

<a id='6'></a>
### 6️⃣ Install latest v2+ Helm ASO chart and include the CRDs

Include (Custom Resource Definitions) you are interested in using. You can use **--set crdPattern=*** to install all the CRDs, but be aware of the limits of the Kubernetes you are running.

In [ ]:
import subprocess  
import json  
import os  
 
# Function to run shell commands  
def run_command(command):  
    print(f"Running command: {command}")  
    try:  
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)  
        return True, output.decode("utf-8")  
    except subprocess.CalledProcessError as e:  
        return False, e.output.decode("utf-8")  
    
service_principal_id = os.environ["AZURE_CLIENT_ID"]

! helm repo add aso2 https://raw.githubusercontent.com/Azure/azure-service-operator/main/v2/charts
! helm upgrade --install aso2 aso2/azure-service-operator --create-namespace --namespace=azureserviceoperator-system --set crdPattern={crd_pattern}

# Once the Azure Service Operator is installed, you can check the status of the pods in the 'azureserviceoperator-system' namespace.
success, pod_output = run_command("kubectl get pods -n azureserviceoperator-system -o json")
if success:
    pods = json.loads(pod_output)  
    pod_name = None  
    for pod in pods["items"]:  
        if "azureserviceoperator-controller-manager" in pod["metadata"]["name"]:  
            pod_name = pod["metadata"]["name"]
            break  
    if pod_name:  
        print(f"Pod name: {pod_name}")
          
        # To view the logs for the running ASO controller, take note of the pod name shown above and then run the following command.
        log_command = f"kubectl logs -n azureserviceoperator-system {pod_name} manager"
        success, logs_output = run_command(log_command)
        if success:
            print(f"Logs:\n{logs_output}")
        else:
            print(f"Failed to retrieve logs. Error:\n{logs_output}")
    else:
        print("No matching pod found.")
else:
    print(f"Failed to get pods. Error:\n{pod_output}")

<a id='7'></a>
### 7️⃣ Create a new Azure Resource with ASO
Let’s create an **Azure Resource Group** in a specified location with the name "aso-sample-rg".

In [ ]:
! kubectl apply -f "rg.yaml"
! kubectl describe resourcegroups/aso-sample-rg